In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('stats.csv')

## Calculating Best Global Heroes

In [ ]:
class best_Heroes:
    def __init__(self, data):
        self.data = data
        self.hero_categories = [
            {"name": "Ana", "category": ["Poke", "Brawl", "Dive"]},
            {"name": "Ashe", "category": ["Poke", "Brawl"]},
            {"name": "Bap", "category": ["Brawl", "Poke"]},
            {"name": "Bastion", "category": ["Poke", "Brawl"]},
            {"name": "Brig", "category": ["Dive"]},
            {"name": "Cass", "category": ["Brawl"]},
            {"name": "DVA", "category": ["Brawl", "Dive"]},
            {"name": "Doom", "category": ["Brawl", "Dive"]},
            {"name": "Echo", "category": ["Dive", "Brawl"]},
            {"name": "Genji", "category": ["Dive", "Brawl", "Poke"]},
            {"name": "Hanzo", "category": ["Poke"]},
            {"name": "Hazard", "category": ["Brawl"]},  
            {"name": "Illari", "category": ["Poke"]},  
            {"name": "Queen", "category": ["Brawl"]},
            {"name": "Junk", "category": ["Brawl"]},
            {"name": "Juno", "category": ["Dive", "Brawl"]},  
            {"name": "Kiri", "category": ["Dive", "Poke", "Brawl"]}, 
            {"name": "LW", "category": ["Dive"]},
            {"name": "Lucio", "category": ["Brawl", "Dive"]}, 
            {"name": "Mauga", "category": ["Brawl"]},
            {"name": "Mei", "category": ["Brawl", "Poke"]}, 
            {"name": "Mercy", "category": ["Poke"]},
            {"name": "Moira", "category": ["Dive", "Brawl"]},
            {"name": "Orisa", "category": ["Brawl", "Poke"]},
            {"name": "Pharah", "category": ["Poke"]},
            {"name": "Ram", "category": ["Brawl", "Poke"]},
            {"name": "Reaper", "category": ["Brawl", "Dive"]},
            {"name": "Rein", "category": ["Brawl"]},
            {"name": "Hog", "category": ["Poke"]},
            {"name": "Sigma", "category": ["Poke", "Brawl"]},
            {"name": "Soj", "category": ["Poke"]},  
            {"name": "Soldier", "category": ["Poke"]},
            {"name": "Sombra", "category": ["Dive"]},
            {"name": "Sym", "category": ["Brawl"]},
            {"name": "Torb", "category": ["Poke", "Brawl"]},  
            {"name": "Tracer", "category": ["Dive"]}, 
            {"name": "Venture", "category": ["Dive", "Brawl"]},  
            {"name": "Widow", "category": ["Poke"]},  
            {"name": "Winston", "category": ["Dive"]},
            {"name": "Ball", "category": ["Dive"]}, 
            {"name": "Zarya", "category": ["Brawl"]},  
            {"name": "Zen", "category": ["Poke"]}  
            ]


    def raw_match_wins(self, mirrored = True):
        hero_winrates: dict[str, list[int]] = {}

        for idx, row in self.data.iterrows():
            result = row['Result']
            allied_heroes = row['My_Tank'].split(',') + row['My_DPS'].split(',') + row['My_Sups'].split(',') 
            enemy_heroes = row['E_Tank'].split(',') + row['E_DPS'].split(',') + row['E_Sups'].split(',')


            all_heroes = set(allied_heroes + enemy_heroes)  # Set for checking duplicates
            
            for hero in all_heroes:
                if not mirrored and hero in allied_heroes and hero in enemy_heroes:
                    continue  

                if hero not in hero_winrates:
                    if hero in row['My_Tank'].split(',') + row['E_Tank'].split(','):
                        role = 'Tank'
                    elif hero in row['My_DPS'].split(',') + row['E_DPS'].split(','):
                        role = 'DPS'
                    elif hero in row['My_Sups'].split(',') + row['E_Sups'].split(','):
                        role = 'Support'
                    else:
                        role = 'Unknown'

                    hero_winrates[hero] = {'matches': 0, 'wins': 0, 'Role': role}

                if hero in allied_heroes:
                    hero_winrates[hero]['matches'] += 1
                    if result == 1:
                        hero_winrates[hero]['wins'] += 1
                if hero in enemy_heroes:
                    hero_winrates[hero]['matches'] += 1
                    if result == 0:
                        hero_winrates[hero]['wins'] += 1
                
        # remove heroes with no matches
        hero_winrates = {hero: stats for hero, stats in hero_winrates.items() if stats['matches'] > 0}

        return hero_winrates

    def get_bayesian_winrate(self, match_wins, a=25, b=25):
        bayesian_winrates: dict[str, list[int]] = {}

        for hero in match_wins:
            matches = match_wins[hero]['matches']
            wins = match_wins[hero]['wins']
            bayesian_winrates[hero] = (wins + a) / (matches + a + b)  # Posterior mean of winrate
        
        return bayesian_winrates
    


    def normalize_winrates(self, match_wins, winrates):
        roles = {'Tank': [], 'DPS': [], 'Support': []}
        # Separate winrates by role
        for hero, winrate in winrates.items():
            role = match_wins[hero]['Role']
            roles[role].append(winrate)
        
        normalized = {}
        
        # Normalize winrates within each role
        for role, winrate_list in roles.items():
            winrate_array = np.array(winrate_list)
            min_winrate = np.min(winrate_array)
            max_winrate = np.max(winrate_array)
            
            for hero, winrate in winrates.items():
                if match_wins[hero]['Role'] == role:
                    normalized[hero] = (winrate - min_winrate) / (max_winrate - min_winrate)
        
        return normalized

    def stat_visualiser(self, values):
        # Separate heroes by role
        roles = {'Tank': [], 'DPS': [], 'Support': []}
        for hero, value in values.items():
            role = match_wins[hero]['Role']
            if value >= 0.75:
                roles[role].append((hero, value))
        
        # Sort each role by value in descending order
        for role in roles:
            roles[role].sort(key=lambda x: x[1], reverse=True)
        
        # Print header
        print(f"{'Tank':<20} {'DPS':<20} {'Support':<20}")
        print("-" * 60)
        
        # Print heroes side by side
        for i in range(max(len(roles['Tank']), len(roles['Support']), len(roles['DPS']))):
            tank = f"{roles['Tank'][i][0]}: {roles['Tank'][i][1]:.4f}" if i < len(roles['Tank']) else ""
            dps = f"{roles['DPS'][i][0]}: {roles['DPS'][i][1]:.4f}" if i < len(roles['DPS']) else ""
            support = f"{roles['Support'][i][0]}: {roles['Support'][i][1]:.4f}" if i < len(roles['Support']) else ""            
            print(f"{tank:<20} {dps:<20} {support:<20}")


    def first_pick(self, teammates, curr_map, comp = "Poke,Brawl,Dive"):
        """ Calculate the total matches and wins of all heroes that were played on a given map or with specific teammates"""

        comp_categories = set(comp.split(","))
        teammates = [hero.strip() for hero in teammates.split(',')] if isinstance(teammates, str) else teammates
        hero_winrates: dict[str, list[int]] = {}

        for idx, row in self.data.iterrows():
            result = row['Result']
            allied_heroes = row['My_Tank'].split(',') + row['My_DPS'].split(',') + row['My_Sups'].split(',')
            enemy_heroes = row['E_Tank'].split(',') + row['E_DPS'].split(',') + row['E_Sups'].split(',')
            my_map = row['Map']

            # Check if any teammate is in allied heroes or if the map matches the current map
            self.synergy(teammates, curr_map, hero_winrates, row, result, allied_heroes, enemy_heroes, my_map, comp_categories)


        for teammate in teammates:
            if teammate in hero_winrates:
                del hero_winrates[teammate]
        return hero_winrates

    def synergy(self, teammates, curr_map, hero_winrates, row, result, allied_heroes, enemy_heroes, my_map, comp_categories):
        if any(teammate in allied_heroes for teammate in teammates) or my_map == curr_map:
            for hero in allied_heroes:
                
                hero_data = next((h for h in self.hero_categories if h["name"] == hero), None)
                if comp_categories.isdisjoint(hero_data['category']):
                    continue
                if hero not in hero_winrates:
                    if hero in row['My_Tank'].split(',') + row['E_Tank'].split(','):
                        role = 'Tank'
                    elif hero in row['My_DPS'].split(',') + row['E_DPS'].split(','):
                        role = 'DPS'
                    elif hero in row['My_Sups'].split(',') + row['E_Sups'].split(','):
                        role = 'Support'
                    else:
                        role = 'Unknown'

                    hero_winrates[hero] = {'matches': 0, 'wins': 0, 'Role': role}
                
                hero_winrates[hero]['matches'] += 1
                if result == 1:
                    hero_winrates[hero]['wins'] += 1

        if any(enemy in enemy_heroes for enemy in teammates) or my_map == curr_map:
            for hero in enemy_heroes:

                hero_data = next((h for h in self.hero_categories if h["name"] == hero), None)
                if comp_categories.isdisjoint(hero_data['category']):
                    continue

                if hero not in hero_winrates:
                    if hero in row['My_Tank'].split(',') + row['E_Tank'].split(','):
                        role = 'Tank'
                    elif hero in row['My_DPS'].split(',') + row['E_DPS'].split(','):
                        role = 'DPS'
                    elif hero in row['My_Sups'].split(',') + row['E_Sups'].split(','):
                        role = 'Support'
                    else:
                        role = 'Unknown'

                    hero_winrates[hero] = {'matches': 0, 'wins': 0, 'Role': role}
                hero_winrates[hero]['matches'] += 1
                if result == 0:
                    hero_winrates[hero]['wins'] += 1
                

heroes = best_Heroes(df)
match_wins = heroes.first_pick("DVA, Soj, Kiri, LW", "Colosseo", "Poke")
print(match_wins)

bayesian_winrates = heroes.get_bayesian_winrate(match_wins)
print(bayesian_winrates)
normalised_winrates = heroes.normalize_winrates(match_wins, bayesian_winrates)
heroes.stat_visualiser(normalised_winrates)

{'Mercy': {'matches': 23, 'wins': 12, 'Role': 'Support'}, 'Ashe': {'matches': 26, 'wins': 11, 'Role': 'DPS'}, 'Ana': {'matches': 52, 'wins': 25, 'Role': 'Support'}, 'Bap': {'matches': 2, 'wins': 0, 'Role': 'Support'}, 'Genji': {'matches': 22, 'wins': 10, 'Role': 'DPS'}, 'Ram': {'matches': 8, 'wins': 5, 'Role': 'Tank'}, 'Orisa': {'matches': 6, 'wins': 2, 'Role': 'Tank'}, 'Soldier': {'matches': 9, 'wins': 5, 'Role': 'DPS'}, 'Torb': {'matches': 14, 'wins': 9, 'Role': 'DPS'}, 'Mei': {'matches': 5, 'wins': 0, 'Role': 'DPS'}, 'Sigma': {'matches': 6, 'wins': 2, 'Role': 'Tank'}, 'Bastion': {'matches': 2, 'wins': 0, 'Role': 'DPS'}, 'Hanzo': {'matches': 2, 'wins': 1, 'Role': 'DPS'}, 'Zen': {'matches': 4, 'wins': 2, 'Role': 'Support'}, 'Pharah': {'matches': 4, 'wins': 3, 'Role': 'DPS'}, 'Illari': {'matches': 6, 'wins': 4, 'Role': 'Support'}, 'Widow': {'matches': 2, 'wins': 1, 'Role': 'DPS'}}
{'Mercy': 0.5068493150684932, 'Ashe': 0.47368421052631576, 'Ana': 0.49019607843137253, 'Bap': 0.4807692307

## Best DPS Hero

In [3]:
import warnings
warnings.filterwarnings("ignore")



class top_dps:
    """ Get top DPS heroes."""

    def __init__(self, my_team, enemy_team, my_map, data):
        self.my_team = my_team
        self.enemy_team = enemy_team
        self.data = data
        self.my_map = my_map


    def my_winrate(self):
        dps_heroes = {
            "Ashe": [0, 0], "Bastion": [0, 0], "Cass": [0, 0], "Echo": [0, 0], "Genji": [0, 0], "Hanzo": [0, 0], "Junk": [0, 0], 
            "Mei": [0, 0], "Pharah": [0, 0], "Reaper": [0, 0], "Soj": [0, 0], "Soldier": [0, 0], "Sombra": [0, 0], "Sym": [0, 0],
            "Torb": [0, 0], "Tracer": [0, 0], "Venture": [0, 0], "Widow": [0, 0]
        }  # matches, wins
        del dps_heroes[my_team['DPS'][0]]  # Remove the DPS hero we are looking at

        for idx, row in self.data.iterrows():
            my_dps = row['My_Hero'].split(',')

            if any(hero in my_dps for hero in dps_heroes):
                for hero in dps_heroes:
                    if hero in my_dps:
                        dps_heroes[hero][0] += 1
                        dps_heroes[hero][1] += row['Result']

        alpha = 0.06  # Controls how fast winrate stabilizes

        # Compute adjusted winrates (weighted by games played)
        adjusted_winrates = {}
        for hero, (matches, wins) in dps_heroes.items():
            if matches > 0:
                raw_winrate = wins / matches
                dampening_factor = 1 - np.exp(-alpha * matches)  # Exponential scaling
                adjusted_winrate = 0.25 + (raw_winrate - 0.25) * dampening_factor
            else:
                adjusted_winrate = 0.25  # Default if no games played

            adjusted_winrates[hero] = adjusted_winrate

        # Normalize adjusted winrates to [0.8, 1.2] scale
        min_wr = min(adjusted_winrates.values())
        max_wr = max(adjusted_winrates.values())

        normalized_winrates = {}
        for hero, wr in adjusted_winrates.items():
            if max_wr > min_wr:  # Avoid division by zero
                normalized_wr = 0.8 + ((wr - min_wr) / (max_wr - min_wr)) * (1.2 - 0.8)
            else:
                normalized_wr = 1.0  # Default neutral multiplier if all winrates are the same
            normalized_winrates[hero] = normalized_wr

        return normalized_winrates





    def base_winrate(self):
        """ Extract Each chosen DPS base winrate. """
        base_heroes = {"Ashe": [0, 0], "Bastion": [0, 0], "Cass": [0, 0], "Echo": [0, 0],  "Genji":  [0, 0], "Hanzo": [0, 0], "Junk": [0, 0], 
                    "Mei": [0, 0], "Pharah": [0, 0], "Reaper":  [0, 0], "Soj":  [0, 0], "Soldier": [0, 0], "Sombra": [0, 0], "Sym": [0, 0],
                    "Torb": [0, 0], "Tracer":  [0, 0], "Venture": [0, 0], "Widow": [0, 0]} # matches, wins
        
        del base_heroes[my_team['DPS'][0]]  # Remove the DPS hero we are looking at
        for idx, row in self.data.iterrows():
            a_dps = row['My_DPS'].split(',')
            e_dps = row['E_DPS'].split(',')
            
            if any(hero in a_dps for hero in base_heroes) or any(hero in e_dps for hero in base_heroes):
                for hero in base_heroes:
                    if hero in a_dps:
                        base_heroes[hero][0] += 1
                        base_heroes[hero][1] += row['Result']
                    if hero in e_dps:
                        base_heroes[hero][0] += 1
                        base_heroes[hero][1] += 1 - row['Result']
        return(base_heroes)
                



    def vs_winrate(self, enemy_team):
        """ Calculate counter winrate for each hero"""
        vs_heroes = {"Ashe": [0, 0], "Bastion": [0, 0], "Cass": [0, 0], "Echo": [0, 0],  "Genji":  [0, 0], "Hanzo": [0, 0], "Junk": [0, 0], 
                    "Mei": [0, 0], "Pharah": [0, 0], "Reaper":  [0, 0], "Soj":  [0, 0], "Soldier": [0, 0], "Sombra": [0, 0], "Sym": [0, 0],
                    "Torb": [0, 0], "Tracer":  [0, 0], "Venture": [0, 0], "Widow": [0, 0]}

        del vs_heroes[my_team['DPS'][0]]  # Remove the DPS hero we are looking at

        for idx, row in self.data.iterrows():
            e_tank = row['E_Tank'].split(',')
            e_dps = row['E_DPS'].split(',')
            e_sups = row['E_Sups'].split(',')
            a_tank = row['My_Tank'].split(',')
            a_dps = row['My_DPS'].split(',')
            a_sups = row['My_Sups'].split(',')

            for hero in vs_heroes:
                if hero in a_dps:
                    # Check if Reaper played against any enemy tank, DPS, or support
                    if any(tank in e_tank for tank in enemy_team['Tank']) or \
                    any(dps in e_dps for dps in enemy_team['DPS']) or \
                    any(sup in e_sups for sup in enemy_team['Support']):
                        vs_heroes[hero][0] += 1  # Increment match count
                        vs_heroes[hero][1] += row['Result']  # Increment wins


                if hero in e_dps:
                    # Check if Reaper played against any allied tank, DPS, or support
                    if any(tank in a_tank for tank in enemy_team['Tank']) or \
                    any(dps in a_dps for dps in enemy_team['DPS']) or \
                    any(sup in a_sups for sup in enemy_team['Support']):
                        vs_heroes[hero][0] += 1  # Increment match count
                        vs_heroes[hero][1] += 1 - row['Result']


        return vs_heroes
        
    def ally_winrate(self, my_team):
        """ Calculate synergy winrate for each hero in the team """ 

        ally_heroes = {"Ashe": [0, 0], "Bastion": [0, 0], "Cass": [0, 0], "Echo": [0, 0],  "Genji":  [0, 0], "Hanzo": [0, 0], "Junk": [0, 0], 
                    "Mei": [0, 0], "Pharah": [0, 0], "Reaper":  [0, 0], "Soj":  [0, 0], "Soldier": [0, 0], "Sombra": [0, 0], "Sym": [0, 0],
                    "Torb": [0, 0], "Tracer":  [0, 0], "Venture": [0, 0], "Widow": [0, 0]}
        
        del ally_heroes[my_team['DPS'][0]]  # Remove the DPS hero we are looking at

        for idx, row in self.data.iterrows():
            e_tank = row['E_Tank'].split(',')
            e_dps = row['E_DPS'].split(',')
            e_sups = row['E_Sups'].split(',')
            a_tank = row['My_Tank'].split(',')
            a_dps = row['My_DPS'].split(',')
            a_sups = row['My_Sups'].split(',')

            for hero in ally_heroes:
                if hero in a_dps:
                    # Check if Reaper played against any enemy tank, DPS, or support
                    if any(tank in a_tank for tank in my_team['Tank']) or \
                    any(dps in a_dps for dps in my_team['DPS']) or \
                    any(sup in a_sups for sup in my_team['Support']):
                        ally_heroes[hero][0] += 1
                        ally_heroes[hero][1] += row['Result']

                if hero in e_dps:
                    # Check if Reaper played against any allied tank, DPS, or support
                    if any(tank in e_tank for tank in my_team['Tank']) or \
                    any(dps in e_dps for dps in my_team['DPS']) or \
                    any(sup in e_sups for sup in my_team['Support']):
                        ally_heroes[hero][0] += 1
                        ally_heroes[hero][1] += 1 - row['Result']

        return ally_heroes
    

    def map_winrate(self, map):
        """ Calculate map winrate for each hero in the data """
        map_heroes = {"Ashe": [0, 0], "Bastion": [0, 0], "Cass": [0, 0], "Echo": [0, 0],  "Genji":  [0, 0], "Hanzo": [0, 0], "Junk": [0, 0], 
                    "Mei": [0, 0], "Pharah": [0, 0], "Reaper":  [0, 0], "Soj":  [0, 0], "Soldier": [0, 0], "Sombra": [0, 0], "Sym": [0, 0],
                    "Torb": [0, 0], "Tracer":  [0, 0], "Venture": [0, 0], "Widow": [0, 0]}
        
        
        del map_heroes[my_team['DPS'][0]]  # Remove the DPS hero we are looking at

        for idx, row in self.data.iterrows():
            e_dps = row['E_DPS'].split(',')
            a_dps = row['My_DPS'].split(',')

            for hero in map_heroes:
                if hero in a_dps:
                    # Check if Reaper played against any enemy tank, DPS, or support
                    if row['Map'] == map:
                        map_heroes[hero][0] += 1  # Increment match count
                        map_heroes[hero][1] += row['Result']

                if hero in e_dps:
                    # Check if Reaper played against any allied tank, DPS, or support
                    if row['Map'] == map:
                        map_heroes[hero][0] += 1
                        map_heroes[hero][1] += 1 - row['Result']

        return map_heroes

    def standardised_winrate(self):
        """ Calculate standardised winrate for each hero in the data """
        base = self.base_winrate()
        vs = self.vs_winrate(self.enemy_team)
        ally = self.ally_winrate(self.my_team)
        my_map = self.map_winrate(self.my_map)
        my_winrate = self.my_winrate()  # Get normalized winrate multipliers
        alpha = 0.06
        baseline = 0.25
        # Create a dictionary to hold winrates
        all_winrates = {hero: [] for hero in base.keys()}

        # Compute raw winrates for each hero in each category
        for hero in base:
            for dataset in [base, vs, ally, my_map]:
                matches, wins = dataset[hero]
                dampening_factor = 1 - np.exp(-alpha * matches)
                winrate = baseline + ((wins / matches) - baseline) * dampening_factor if matches >= 5 else baseline

                all_winrates[hero].append(winrate)
        # Convert to NumPy array for easier computation
        winrate_matrix = np.array(list(all_winrates.values()))
        # Compute mean and standard deviation along columns
        mean_winrate = np.mean(winrate_matrix, axis=0)
        std_winrate = np.std(winrate_matrix, axis=0)

        # Standardize winrates
        standardized = {}
        for i, hero in enumerate(base.keys()):
            winrates = (winrate_matrix[i] - mean_winrate) / std_winrate
            # Compute the standardized score based on differences
            base_wr, vs_wr, ally_wr, map_wr = winrates  # Unpack standardized winrates

            base_wr = 0 if np.isnan(base_wr) else base_wr
            vs_wr = base_wr if np.isnan(vs_wr) else vs_wr
            ally_wr = base_wr if np.isnan(ally_wr) else ally_wr
            map_wr = base_wr if np.isnan(map_wr) else map_wr

            standardized_score = base_wr + ((vs_wr - base_wr) + (ally_wr - base_wr) + map_wr) if map_wr > 0 else base_wr + ((vs_wr - base_wr) + (ally_wr - base_wr))           


            # Apply personal winrate multiplier
            if hero in my_winrate:
                if standardized_score > 0:
                    standardized_score *= my_winrate[hero]  # Multiply by normalized winrate only for positive scores
                else:
                    standardized_score *= 1 + (1 - my_winrate[hero])  # If negative, apply a reduced multiplier or no multiplier

            standardized[hero] = standardized_score




            
        # Normalize the standardized winrates using Min-Max scaling
        min_score = min(standardized.values())
        max_score = max(standardized.values())

        normalized_standardized = {hero: (score - min_score) / (max_score - min_score) for hero, score in standardized.items()}

        # Sort and return top 3 heroes
        sorted_winrates = sorted(normalized_standardized.items(), key=lambda x: x[1], reverse=True)

        # Convert np.float64 to standard Python float
        cleaned_winrates = [(hero, float(winrate)) for hero, winrate in sorted_winrates]

        return cleaned_winrates

       
        

my_team = {
    'Tank': ['Zarya'],   # Your team's tank(s)
    'DPS': ['Genji'],    # Your selected DPS
    'Support': ['Ana', 'Moira']  # Your team's support(s)
}
    
enemy_team = {
    'Tank': ['rein'],   # Enemy team's tank(s)
    'DPS': ['Bastion', 'Ashe'],  # Enemy team's DPS
    'Support': ['Kiri', 'Bap']  # Enemy team's support(s)
}

my_map = "Havana"





df = pd.read_csv('stats.csv')
df = df[["My_Hero", "My_Tank", "My_DPS", "My_Sups", "E_Tank", "E_DPS", "E_Sups", "Result", "Map", "Closeness"]]
dps = top_dps(my_team, enemy_team, my_map, df)




cleaned_winrates = dps.standardised_winrate()

 # Neat printing of the results
print(f"{'Hero':<20} {'Normalized Standardized Winrate':<25}")
print("-" * 55)

for hero, winrate in cleaned_winrates:
    if winrate > 0.9:
        print(f"{hero:<20} {winrate:<25.4f}")


Hero                 Normalized Standardized Winrate
-------------------------------------------------------
Torb                 1.0000                   
